<a href="https://colab.research.google.com/github/GretaDanesi18/Olive_Detection_Tracking/blob/main/source_detector/YOLOv5_SDet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **SETUP**

Per prima cosa importiamo le librerie necessarie

In [1]:
import torch 
import os
import random 
import shutil
import glob
from IPython.display import Image,display
from PIL import Image, ImageDraw
import cv2
import pandas as pd
import subprocess
from numpy import source


Controlliamo l'utilizzo del device GPU oppure CPU

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print('Using PyTorch version:', torch.__version__, ' Device:', device)

Successivamente cloniamo la repo contente YOLOv5 e installiamo i requisiti

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

## **CREAZIONE DATASET SOURCE DETECTOR**

Cloniamo la repo contente il dataset iniziale, i frame dei video e il file yaml

In [ ]:
!git clone https://github.com/GretaDanesi18/Olive_Detection_Tracking.git

Attraverso la libreria os creiamo le directory in cui inserire le labels e le immagini necessarie a creare il dataset per yolov5


```
/path/to/dataset/...

+--images
|   +---val
         +-----image_1.jpg
         +-----image_2.jpg
         +-----image_n.jpg
|   +---train
        +-----image_1.jpg
        +-----image_2.jpg
        +-----image_n.jpg
+--labels
|   +---val
         +-----label_1.txt
         +-----label_2.txt
         +-----label_n.txt
|   +---train
        +-----label_1.txt
        +-----label_2.txt
        +-----label_n.txt

```


In [5]:
try:
  if not os.path.exists('/content/yolov5/Olive_Detection_Tracking/images/val'):
    os.makedirs('/content/yolov5/Olive_Detection_Tracking/images/val')

except OSError:
    print('Errore nella creazione della cartella images/val')

try:
  if not os.path.exists('/content/yolov5/Olive_Detection_Tracking/images/train'):
    os.makedirs('/content/yolov5/Olive_Detection_Tracking/images/train')

except OSError:
    print('Errore nella creazione della cartella images/train')

try:
  if not os.path.exists('/content/yolov5/Olive_Detection_Tracking/labels/val'):
    os.makedirs('/content/yolov5/Olive_Detection_Tracking/labels/val')

except OSError:
    print('Errore nella creazione della cartella labels/val')

try:
  if not os.path.exists('/content/yolov5/Olive_Detection_Tracking/labels/train'):
    os.makedirs('/content/yolov5/Olive_Detection_Tracking/labels/train')

except OSError:
    print('Errore nella creazione della cartella labels/train')


Attraverso degli array di appoggio dividiamo le labels dalle immagini e settiamo le percentuali con cui fare lo split del dataset

In [ ]:

#path
images_path = '/content/yolov5/Olive_Detection_Tracking/source_detector/dataset_source/images'
labels_path = '/content/yolov5/Olive_Detection_Tracking/source_detector/dataset_source/labels'
train_image_path = '/content/yolov5/Olive_Detection_Tracking/images/train'
val_image_path = '/content/yolov5/Olive_Detection_Tracking/images/val'
train_label_path ='/content/yolov5/Olive_Detection_Tracking/labels/train'
val_label_path = '/content/yolov5/Olive_Detection_Tracking/labels/val'


#percentuale per train, val
val_ratio = 0.2
train_ratio = 0.8

lista_label = sorted(os.listdir(labels_path))
lista_image = sorted(os.listdir(images_path))

#numero di immagini per train e per val
num_train = int(len(lista_image)*train_ratio)
num_val = int(len(lista_image)*val_ratio)

print("Immagini per il training: ", num_train)
print("Immagini per validation: ", num_val)



Con la funzione random separiamo in modo casuale le immagini usate per il training da quelle usate per la validation.

Successivamente inseriamo le rispettive labels nella directory per il training oppure per la validation

In [7]:
#inserisco image e label in train
for i in range(num_train):
   file_image = random.choice(lista_image)
   file_image_name = file_image.split('.')[0]
   file_label =  file_image_name+'.txt'

   #sposto i file nella cartella train
   source_label = labels_path+'/'+file_label
   destination_label = train_label_path+'/'+file_label
   shutil.copyfile(source_label,destination_label)

   source_image = images_path+'/'+file_image
   destination_image = train_image_path+'/'+file_image
   shutil.copyfile(source_image,destination_image)

   #rimuovo i file dalla lista
   lista_image.remove(file_image)
   lista_label.remove(file_label)
    

#inserisco image e label in val  
for i in range(num_val):
    file_image =random.choice(lista_image)
    file_image_name = file_image.split('.')[0]
    file_label = file_image_name+'.txt'

    #sposto i file nella cartella val
    source_label = labels_path+'/'+file_label
    destination_label = val_label_path+'/'+file_label
    shutil.copyfile(source_label,destination_label)

    source_image = images_path+'/'+file_image
    destination_image = val_image_path+'/'+file_image
    shutil.copyfile(source_image,destination_image)

    #rimuovo i file dalla lista
    lista_image.remove(file_image)
    lista_label.remove(file_label)

Inseriamo file yaml nella directory data

In [ ]:
source = 'Olive_Detection_Tracking/source_detector/source_dataset.yaml'
destination = 'data/source_dataset.yaml'
shutil.copy(source,destination)

## **ADDESTRAMENTO SOURCE DETECTOR**

Usiamo Weights & Biases per mostrare i risultati.

Eventualmente possiamo usare anche tensorboard lanciando il seguente codice dopo aver avviato il training.

```
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs
```



In [ ]:
#Weights & Biases
%pip install -q wandb
import wandb
wandb.login()


Avviamo l'addestramento 

In [ ]:
!python train.py --img 1280 --batch 16 --epochs 80 --data source_dataset.yaml --weights yolov5s.pt --cache

Controlliamo i risultati stampandoli a schermo

Modificare i path a seconda di quali risultati si vuole visualizzare

In [ ]:
Image.open('runs/train/exp/train_batch0.jpg')

In [ ]:
Image.open('runs/train/exp/val_batch0_labels.jpg')

In [ ]:
Image.open('runs/train/exp/val_batch0_pred.jpg')

In [ ]:

file_csv=pd.read_csv('runs/train/exp/results.csv')
file_csv.head(-1)


In [ ]:
Image.open('runs/train/exp/results.png')

## **VALIDATION SOURCE DETECTOR**

Dopo aver fatto il training facciamo la validation sul dateset del test

In [12]:
!python val.py --img 1280 --batch 16 --data source_dataset.yaml --weights runs/train/exp/weights/best.pt --task test --save-txt

val: data=/content/yolov5/data/source_dataset.yaml, weights=['runs/train/exp/weights/best.pt'], batch_size=16, imgsz=1280, conf_thres=0.001, iou_thres=0.6, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=True, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-307-g92e47b8 Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning '/content/yolov5/../yolov5/Olive_Detection_Tracking/test_dataset/labels' images and labels...100 found, 0 missing, 0 empty, 0 corrupt: 100% 100/100 [00:00<00:00, 362.27it/s]
test: New cache created: /content/yolov5/../yolov5/Olive_Detection_Tracking/test_dataset/labels.cache
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 7/7 [00:12<00:00,  1.81s/it]
                 all        100       3

## **INFERENZA SOURCE DETECTOR**




Facciamo il download del weights migliore cosi da salvarlo in locale e poterlo riutilizzare per inferences future

Modificare i path in accordo con le proprie directory

In [ ]:
from google.colab import files
files.download('runs/train/exp/weights/best.pt')

Facciamo l'inferenza, tramite detect.py, sui frame del video1 cosi da ottenere delle labels che poi verrano usate per generare le pseudo labels.

L'inferenza viene fatta con una threshold di 0.70.
 
I risultati sono salvati in runs/detect/exp

In [ ]:
!python detect.py --source Olive_Detection_Tracking/dataset_video/frame_video/frame_video1 --data data/source_dataset.yaml --img 1280 --weights runs/train/exp/weights/best.pt --save-txt --conf-thres 0.70

Facciamo l'inferenza anche sui frame del video2 per poi poter confrontare i risultati con i bounding box generati dal target detector

In [ ]:
!python detect.py --source Olive_Detection_Tracking/dataset_video/frame_video/frame_video2 --data data/source_dataset.yaml --img 1280 --weights runs/train/exp/weights/best.pt --save-txt --conf-thres 0.40

Se vogliamo salvare soltanto le labels creiamo la zip della cartella contenente le labels cosi da poter fare il download e salvarle in locale.

In [ ]:
!zip -r source_detection_labels.zip runs/detect/exp/labels

In [ ]:
from google.colab import files
files.download("source_detection_labels.zip")


Per salvare sia le labels che le immagini facciamo il download di tutta la cartella in cui sono salvati i risultati runs/detect/exp 

In [ ]:
!zip -r source_detection_results.zip runs/detect/exp

In [ ]:
from google.colab import files
files.download("source_detection_results.zip")

Per salvare tutti i risultati

In [ ]:
!zip -r source_results.zip runs

In [ ]:
from google.colab import files
files.download("source_results.zip")